# Automatiser des tâches Excel avec Python via XLWings

XLWings est une bibliothèque Python qui permet d'interagir avec Microsoft Excel en utilisant le modèle d'objet Excel. Cela signifie que XLWings utilise les objets et méthodes d'Excel pour effectuer des opérations telles que la lecture et l'écriture de données, la manipulation de feuilles de calcul, la création de graphiques, etc.

Fondamentalement, XLWings a créé une interface qui permet de passer des objets Excel (app, book, sheets, range) aux objets Python et vice-versa. 

<img src="modele objet excel.png" width="700"/>



## Les bases d'XLWings

In [72]:
import xlwings as xw

#### Ouvrir un nouveau classeur

In [73]:
wb0 = xw.Book()       

#### Ouvrir un classeur Excel existant

In [74]:
wb1 = xw.Book('mon_classeur.xlsx')

#### Accéder à une feuille spécifique

In [75]:
feuille = wb1.sheets['Tabelle1']

#### Lire une valeur d'une cellule

In [76]:
valeur = feuille.range('A1').value
print(valeur)

None


#### Ajouter une valeur spécifique

In [77]:
feuille.range('B1').value = 'World'

#### Créer un graphique

In [78]:
chart = feuille.charts.add()
chart.set_source_data(feuille.range('A1:B10'))
chart.chart_type = 'line'

## Cas d'application : automatiser la réalisation de business reports

Dans le dossier `s3_data`, vous avez toutes les données de vente d'une chaine de magasin. Votre supérieur veut que vous produisiez un rapport d'analyse de ces ventes, en restant sur Excel. 
L'objectif est ici d'arriver à créer un Jupyter notebook qui va automatiser : 

* L'agrégation de tous les fichiers en une seule base de données
* L'analyse des données
* La création de graphiques 

### Automatiser l'agrégation des fichiers

In [79]:
from pathlib import Path
import pandas as pd

Nous allons d'abord créer une variable this_dir qui contient le chemin absolu du répertoire dans lequel ce Jupyter notebook est actuellement exécuté. Cette variable va permettre au programme de construire les chemins qui permettent de localiser tous les fichiers Excel. 

In [80]:
this_dir = Path().resolve()
print(this_dir)

C:\Users\julia\OneDrive\Bureau\PythonForExcel


Nous allons ensuite initialiser une liste vide appelée `parts` dans laquelle nous allons vouloir stocker tous les dataframes issus des fichiers Excel. 

Pour cela, nous allons demander au programmer de : 
* parcourir tous les fichiers avec une extension ".xls" ou ".xlsx" dans le répertoire "s3_data" situé dans le répertoire actuel
* ouvrir les fichiers excel et les transformer en dataframes
* ajouter les dataframes à la liste vide

In [81]:
parts = []
for path in (this_dir / "s3_data").rglob("*.xls*"):
    print(f'Reading {path.name}')
    part = pd.read_excel(path, index_col="transaction_id")
    parts.append(part)

Reading April.xlsx
Reading August.xlsx
Reading December.xlsx
Reading February.xlsx
Reading January.xlsx
Reading July.xlsx
Reading June.xlsx
Reading March.xlsx
Reading May.xlsx
Reading November.xlsx
Reading October.xlsx
Reading September.xlsx


In [82]:
df = pd.concat(parts)
df

,store,status,transaction_date,plan,contract_type,amount
transaction_id,,,,,,
7fb801a5,Chicago,ACTIVE,2019-04-01,Silver,NEW,14.25
8e8bdb2a,San Francisco,ACTIVE,2019-04-01,Silver,NEW,14.25
239db9b6,Washington DC,ACTIVE,2019-04-01,Silver,NEW,14.25
7ccb13ab,San Francisco,ACTIVE,2019-04-01,Gold,NEW,19.35
c18e82d9,Chicago,ACTIVE,2019-04-01,Bronze,NEW,12.20
...,...,...,...,...,...,...
53a449b0,New York,ACTIVE,2019-09-30,Silver,NEW,14.25
35b46c26,Boston,ACTIVE,2019-09-30,Silver,NEW,14.25
9e498c85,Chicago,ACTIVE,2019-09-30,Silver,NEW,14.25


Commençons par créer une feuille dans notre fichier Excel

In [83]:
database = wb1.sheets['Database']

Et ajoutons tout le dataframe dedans.

In [84]:
database["A1"].value = df

### Automatiser l'analyse

Nous voulons ici avoir le total des ventes par magasin et par mois

In [85]:
pivot = pd.pivot_table(df, index = 'transaction_date', columns='store',values="amount", aggfunc="sum")

In [86]:
summary = pivot.resample("ME").sum()
summary.index.name = "Month"

In [87]:
print(summary)

store         Boston   Chicago  Las Vegas  New York  San Francisco  \
Month                                                                
2019-01-31  13701.10  26272.65    7216.45  42128.25       42969.25   
2019-02-28  12243.85  24739.90    6192.60  37926.55       36469.35   
2019-03-31  11026.60  22131.00    5637.25  32805.60       33876.45   
2019-04-30  10344.80  21489.35    5540.40  32954.35       33265.00   
2019-05-31  10533.15  20606.55    5266.45  32348.55       31836.80   
2019-06-30  13317.90  24328.15    5713.55  38543.65       37787.30   
2019-07-31  10701.65  20877.90    5376.65  31547.60       31386.65   
2019-08-31  12239.70  24662.75    6335.05  36491.60       36895.10   
2019-09-30  10757.10  20964.35    5928.55  33226.20       31709.80   
2019-10-31   8180.35  16446.30    4217.25  24889.95       26015.90   
2019-11-30  12583.90  25794.75    6880.75  38646.20       38479.20   
2019-12-31  10390.00  21619.95    5481.60  32828.10       32378.60   

store       Washing

Ajoutons maintenant cette analyse dans le tableur Excel

In [88]:
summary_sheet = wb1.sheets['Summary']

In [89]:
summary_sheet["A1"].value = summary


### Automatiser la réalisation de graphiques

In [90]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(summary, label=summary.columns)

plt.legend(loc='upper left')

summary_sheet.pictures.add(fig, name='MyPlot', update=True, left=summary_sheet.range('B16').left, top=summary_sheet.range('B16').top)

<Picture 'MyPlot' in <Sheet [mon_classeur.xlsx]Summary>>